In [1]:
#vengono importate le librerie necessarie

import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
import keras_metrics as km
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

Using TensorFlow backend.


In [2]:
#la funzione read_dataset legge il file contenente il dataset e riceve come input il nome del file

def read_dataset(file_name):
    df=pd.read_csv("/home/davide/Scrivania/Advanced Machine Learning/Assignment 1/{}".format(file_name))
    return df

In [3]:
#la funzione split_dataset divide il dataset in features e labels. Essa richiama prima di tutto la funzione 
#read_dataset e poi effettua lo split. Questa funzione prende come parametri di input il nome del file e il nome
#della colonna di target.

def split_dataset(file_name,target):
    df=read_dataset(file_name)
    labels=df["{}".format(target)]
    dataset=df.drop(["{}".format(target)],axis=1)
    return dataset,labels

In [4]:
# prima di costruire la rete neurale viene fatto un minimo di preprocessing per capire quali siano le features 
#più importanti ai fini di una corretta classificazione. La funzione prepare_data restituisce un nuovo dataframe
#con le features standardizzate (X_std) e il vettore di label (y). Questa funzione effettua come prima cosa
#lo split del dataset tramite la funzione definita precedentemente e poi effettua la standardizzazione.
#Riceve come parametri il nome del file e il nome della colonna di target.

def prepare_data(file_name,target):
    X,y=split_dataset(file_name,target)
    scaler=StandardScaler()
    X_std=scaler.fit_transform(X)
    X_std=pd.DataFrame(X_std,columns=X.columns)
    return X_std,y

In [5]:
#La funzione univariate_selection effettua una feature selection univariata. Questa scelta è derivata dalla volontà 
#che l'importanza delle features non fosse influenzata dal tipo di classificatore utilizzato.
#Questa funzione prende come parametri il nome del file e il nome della colonna di target. Viene prima letto 
#il dataset tramite la funzione read_dataset, viene poi invocata la funzione prepare_data e poi vengono stampate 
#in ordine di importanza le features che sono maggiormente significative rispetto alla variabile di target.

def univariate_selection(file_name,target):
    df=read_dataset(file_name)
    X,y=prepare_data(file_name,target)
    
    #parametri di default (score_func=f_classif,k=10) 
    best_features=SelectKBest()
    fit=best_features.fit(X,y)
    
    df_scores = pd.DataFrame(fit.scores_)
    df_columns = pd.DataFrame(X.columns)
    feature_scores = pd.concat([df_columns,df_scores],axis=1)
    feature_scores.columns = ['Specs','Score']
    
    print(feature_scores.nlargest(len(df.columns),'Score'))

In [6]:
# di seguito vengono stampate le features in ordine di importanza. Si è deciso di prendere in considerazione solo 
#le prime 8.

univariate_selection("train.csv","default.payment.next.month")

        Specs        Score
5       PAY_0  3188.598395
6       PAY_2  1959.137288
7       PAY_3  1562.739676
8       PAY_4  1318.868708
9       PAY_5  1167.201233
10      PAY_6   985.403368
0   LIMIT_BAL   646.848436
17   PAY_AMT1   146.980092
18   PAY_AMT2    85.641326
21   PAY_AMT5    84.843470
20   PAY_AMT4    83.914118
19   PAY_AMT3    79.029125
22   PAY_AMT6    71.249181
1         SEX    45.878062
3    MARRIAGE    18.595389
2   EDUCATION    18.328189
11  BILL_AMT1    10.119373
12  BILL_AMT2     5.202868
13  BILL_AMT3     4.963755
4         AGE     4.545373
14  BILL_AMT4     2.717445
15  BILL_AMT5     0.991987
16  BILL_AMT6     0.564829


In [7]:
#la funzione get_important_variables serve per estrarre le feautures importanti basate sull'osservazione del chunk 
#di cui sopra. Questa funzione prende in input il nome del file e una lista di parametri da selezionare 
#dal dataset che viene letto.

def get_important_variables(file_name,parameter_list):
    df=read_dataset(file_name)
    new_df=pd.DataFrame()
    for i in range(len(parameter_list)):
        new_df["{}".format(parameter_list[i])]=df["{}".format(parameter_list[i])]
        
    return new_df

In [8]:
#la funzione preprocess_dataset prepara il dataset per poter essere usato dalla rete neurale. Prima di tutto
#vengono selezionate le variabili significative tramite la funzione get_important_variables, poi vengono
#standardizzate le features e le labels vengono trasformate con il processo di one-hot encoding. 
#Vengono poi restituite le features e le labels processate. Questa funzione prende come parametri di input il nome
#del file, la lista di parametri da selezionare e il numero di classi di output per la classificazione finale. 

def preprocess_dataset(file_name,parameter_list,target,num_classes):
    new_df=get_important_variables(file_name,parameter_list)
    y=new_df["{}".format(target)]
    X=new_df.drop(["{}".format(target)],axis=1)
    scaler=StandardScaler()
    X=scaler.fit_transform(X)
    y=np_utils.to_categorical(y,num_classes=num_classes)
    return X,y

In [9]:
#viene splittato il dataset tra le features e le labels, selezionando preliminarmente solo le features importanti. 

X,y=preprocess_dataset("train.csv",["PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","LIMIT_BAL","PAY_AMT1",
                                    "default.payment.next.month"],"default.payment.next.month",2)

In [10]:
#le features e le labels vengono splittate in train e test,secondo i parametri passati

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.15,random_state=42)

In [11]:
#vengono riconvertiti in un array i valori delle labels del training set, per poter poi essere passate 
#correttamente alla funzione di GridSearch

y_true_train=[]
for i in range(len(y_train)):
    y_true_train.append(np.argmax(y_train[i]))

In [301]:
#viene creata la la funzione create_model, a cui viene passato come input una lista di layer, che la funzione 
#di GridSearch dovrà ottimizzare.

def create_model(layers):
    
    model=Sequential()
    
    for i,neurons in enumerate(layers):
        if i==0:
            model.add(Dense(neurons,input_shape=(8,),activation='relu'))
        else:
            model.add(Dense(neurons,activation='relu'))
            
    model.add(Dense(2,activation='softmax'))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy')
    
    return model


model=KerasClassifier(build_fn=create_model,batch_size=32,epochs=50,verbose=0)

layers=[[8,16],[16,32],[8,16,32],[16,32,64]]

param_grid=dict(layers=layers)

grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=3,scoring='f1')
grid_result=grid.fit(X_train,y_true_train,class_weight={0:1,1:3.56})

print(grid_result.best_score_,grid_result.best_params_)

0.5197984908619067 {'layers': [8, 16]}


In [12]:
#questa funzione costruisce il modello vero e proprio in base ai parametri restituiti dalla grid search.
#Prende in input le feautures e le labels, per poterne ottenere la shape

def build_model(train_data,train_labels):
    
    model=Sequential()
    model.add(Dense(8,input_shape=(train_data.shape[1],),activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(train_labels.shape[1],activation='softmax'))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy')
    
    return model

In [13]:
#viene costruito il modello e successivamente fittato con il training set e contemporaneamente validato sul 
#validation set. Si è scelta una batch size di 32, e si è dato un peso alle classi. In particolare pesi unitario 
#alla classe 1 (quella maggioritaria) e peso 3.56 alla classe 1 (quella minoritaria), visto il rapporto di class 
#imbalance

model=build_model(X_train,y_train)
model.fit(X_train,y_train,validation_data=(X_val,y_val),batch_size=32,epochs=50,class_weight={0:1,1:3.56})

Train on 22950 samples, validate on 4050 samples
Epoch 1/50
22950/22950 [==============================] - 1s 42us/step - loss: 0.9736 - val_loss: 0.5994
Epoch 2/50
22950/22950 [==============================] - 1s 37us/step - loss: 0.9254 - val_loss: 0.5871
Epoch 3/50
22950/22950 [==============================] - 1s 38us/step - loss: 0.9168 - val_loss: 0.5522
Epoch 4/50
22950/22950 [==============================] - 1s 33us/step - loss: 0.9117 - val_loss: 0.5852
Epoch 5/50
22950/22950 [==============================] - 1s 33us/step - loss: 0.9086 - val_loss: 0.5264
Epoch 6/50
22950/22950 [==============================] - 1s 33us/step - loss: 0.9061 - val_loss: 0.5451
Epoch 7/50
22950/22950 [==============================] - 1s 33us/step - loss: 0.9036 - val_loss: 0.5658
Epoch 8/50
22950/22950 [==============================] - 1s 38us/step - loss: 0.9014 - val_loss: 0.5290
Epoch 9/50
22950/22950 [==============================] - 1s 33us/step - loss: 0.9000 - val_loss: 0.5634
Epoch 

In [304]:
#vengono riconvertite le labels del validation set in un array, per poter poi essere passate correttamente alla
#funzione classification_report.

y_true_pred=[]
for i in range(len(y_val)):
    y_true_pred.append(np.argmax(y_val[i]))
    
y_true_pred=np.array(y_true_pred)

In [305]:
#vengono predette le classi del validation set.

y_pred=model.predict_classes(X_val)

In [306]:
#viene stampato il classification report del validation set.

print(classification_report(y_true_pred,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      3117
           1       0.50      0.62      0.55       933

    accuracy                           0.77      4050
   macro avg       0.69      0.72      0.70      4050
weighted avg       0.79      0.77      0.78      4050



In [307]:
#vengono predette le classi del train_set e viene poi stampato il classification report del training set. 
#Questa operazione viene fatta con lo scopo di verificare che non ci sia (troppo) overfitting tra training set 
#e validation set.

y_pred_train=new_model.predict_classes(X_train)
print(classification_report(y_true_train,y_pred_train))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85     17910
           1       0.49      0.60      0.54      5040

    accuracy                           0.78     22950
   macro avg       0.69      0.71      0.70     22950
weighted avg       0.80      0.78      0.78     22950



In [14]:
#viene importato il test set e vengono selezionate solo le variabili più importanti

df_test=get_important_variables("test.csv",["PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","LIMIT_BAL",
                                            "PAY_AMT1"])

In [15]:
#vengono normalizzate le features

scaler_test=StandardScaler()
X_test=scaler_test.fit_transform(df_test)

In [16]:
#vengono predette le classi del test set

y_test_pred=model.predict_classes(X_test)

In [19]:
#vengono scritte le predizioni su un file di testo

output_file = open("/home/davide/Scrivania/Advanced Machine Learning/Assignment 1/Davide_Sangalli_848013_score1.txt","w")

for i in y_test_pred:
    output_file.write(str(i)+'\n')

output_file.close()